## 7. multi_agent(google_adk)

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.agents.invocation_context import InvocationContext
from google.adk.events import Event
from typing import AsyncGenerator

### 1. 메인함수 구현: 계층적 에이전트 구조

In [3]:
class TaskExecutor(BaseAgent):
    """사용자 정의(비 LLM) 동작을 수행하는 특화 에이전트입니다."""
    name: str = "TaskExecutor"
    description: str = "미리 정의된 작업을 실행합니다."
    async def _run_async_impl(self, context: InvocationContext) -> AsyncGenerator[Event, None]:
        """작업에 대한 사용자 정의 구현 로직입니다."""
        yield Event(author=self.name, content="작업이 성공적으로 완료되었습니다.")

greeter = LlmAgent(
    name="Greeter",
    model="gemini-2.0-flash-exp",
    instruction="당신은 친절하게 인사하는 에이전트입니다."
)

task_doer = TaskExecutor()

coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.0-flash-exp",
    instruction="당신은 Coordinator입니다. 인사는 'Greeter'에게, 작업 실행은 'TaskExecutor'에게 위임하세요.",
    description="인사 담당과 작업 실행기 에이전트 사이의 작업을 조정합니다.",
    sub_agents=[greeter, task_doer]
)

In [4]:
assert greeter.parent_agent == coordinator
assert task_doer.parent_agent == coordinator
print("Agent hierarchy created successfully.")

Agent hierarchy created successfully.


### 2. 두번쨰 예제

- LoopAgent를 사용하여 반복 워크플로우 구축하는 방법
- 코드는 두 에이전트를 정의합니다: ConditionChecker와 ProcessingStep. 
- ConditionChecker는 세션 상태에서 "status" 값을 확인하는 커스텀 에이전트입니다. 
  - "status"가 "completed"이면 ConditionChecker는 루프를 중지하기 위해 이벤트를 에스컬레이션합니다. 
  - 그렇지 않으면 루프를 계속하기 위해 이벤트를 생성합니다. 
- ProcessingStep은 "gemini-2.0-flash-exp" 모델을 사용하는 LlmAgent입니다. 
- 지시사항은 작업을 수행하고 최종 단계라면 세션 "status"를 "completed"로 설정하는 것입니다. 
- 그리고 StatusPoller라는 LoopAgent가 생성됩니다. 
  - StatusPoller는 max_iterations=10으로 구성됩니다. 
  - StatusPoller는 ProcessingStep과 ConditionChecker 인스턴스를 모두 하위 에이전트로 포함합니다
  - LoopAgent는 ConditionChecker가 status가 "completed"임을 발견하거나 10회 반복에 도달할 때까지 하위 에이전트를 순차적으로 실행합니다.

In [5]:
import asyncio
from typing import AsyncGenerator
from google.adk.agents import LoopAgent, LlmAgent, BaseAgent
from google.adk.events import Event, EventActions
from google.adk.agents.invocation_context import InvocationContext

In [6]:
class ConditionChecker(BaseAgent):
    """A custom agent that checks for a 'completed' status in the session state."""
    name: str = "ConditionChecker"
    description: str = "Checks if a process is complete and signals the loop to stop."
    async def _run_async_impl(self, context: InvocationContext) -> AsyncGenerator[Event, None]:
        """Checks state and yields an event to either continue or stop the loop."""
        status = context.session.state.get("status", "pending")
        is_done = (status == "completed")
        if is_done:
            yield Event(author=self.name, actions=EventActions(escalate=True))
        else:
            yield Event(author=self.name, content="Condition not met, continuing loop.")

In [7]:
process_step = LlmAgent(
    name="ProcessingStep",
    model="gemini-2.0-flash-exp",
    instruction="""You are a processing agent.
Perform a task and set the session state 'status' to 'completed' if this is the final step.
""",
    description="Performs processing tasks in a loop.",
)

status_poller = LoopAgent(
    name="StatusPoller",
    max_iterations=10,
    sub_agents=[
        process_step,
        ConditionChecker()
    ]
)

### 3. 세번째: 선형 워크 플로우

In [8]:
from google.adk.agents import SequentialAgent, Agent

step1 = Agent(name="Step1_Fetch", output_key="data")

step2 = Agent(
    name="Step2_Process",
    instruction="Analyze the information found in state['data'] and provide a summary."
)

pipeline = SequentialAgent(
    name="MyPipeline",
    sub_agents=[step1, step2]
)

### 4. 네번째 병렬 에이전트

In [9]:
from google.adk.agents import Agent, ParallelAgent

weather_fetcher = Agent(
    name="weather_fetcher",
    model="gemini-2.0-flash-exp",
    instruction="Fetch the weather for the given location and return only the weather report.",
    output_key="weather_data"
)

news_fetcher = Agent(
    name="news_fetcher",
    model="gemini-2.0-flash-exp",
    instruction="Fetch the top news story for the given topic and return only that story.",
    output_key="news_data"
)

data_gatherer = ParallelAgent(
    name="data_gatherer",
    sub_agents=[
        weather_fetcher,
        news_fetcher
    ]
)

### 5. 다섯번째: 에이전트 as a tool
- 함수 호출과 유사한 방식으로 한 에이전트가 다른 에이전트의 역량을 활용할 수 있게 합니다. 
- 구체적으로 코드는 Google의 LlmAgent와 AgentTool 클래스를 사용하여 이미지 생성 시스템을 정의합니다. 
- 이 구조는 두 에이전트로 구성됩니다: 
    - 부모 artist_agent와 하위 에이전트 image_generator_agent. 
    - generate_image 함수는 이미지 생성을 시뮬레이션하는 간단한 도구로, 모의 이미지 데이터를 반환합니다. 
    - image_generator_agent는 받은 텍스트 프롬프트를 바탕으로 이 도구를 사용할 책임이 있습니다. 
    - artist_agent의 역할은 먼저 창의적인 이미지 프롬프트를 만드는 것입니다. 
    - 그런 다음 AgentTool 래퍼를 통해 image_generator_agent를 호출합니다. 
    - AgentTool은 한 에이전트가 다른 에이전트를 도구로 사용할 수 있도록 하는 브리지 역할을 합니다. 
    - artist_agent가 image_tool을 호출하면 AgentTool은 artist가 만든 프롬프트로 image_generator_agent를 호출합니다. 
    - image_generator_agent는 해당 프롬프트로 generate_image 함수를 사용합니다. 
    
- 최종적으로 생성된 이미지(또는 모의 데이터)가 에이전트를 통해 반환됩니다. 
- 이 아키텍처는 상위 레벨 에이전트가 하위 레벨의 전문화된 에이전트를 조율하여 작업을 수행하는 계층형 에이전트 시스템을 보여줍니다.

In [10]:
from google.adk.agents import LlmAgent
from google.adk.tools import agent_tool
from google.genai import types

In [11]:
def generate_image(prompt: str) -> dict:
    """
    텍스트 프롬프트를 기반으로 이미지를 생성합니다.
    인자:
        prompt: 생성할 이미지에 대한 상세한 설명.
    반환:
        상태와 생성된 이미지 바이트를 포함한 딕셔너리.
    """
    print(f"TOOL: 프롬프트: '{prompt}' 에 대한 이미지 생성 중")
    mock_image_bytes = b"mock_image_data_for_a_cat_wearing_a_hat"
    return {
        "status": "success",
        "image_bytes": mock_image_bytes,
        "mime_type": "image/png"
    }

image_generator_agent = LlmAgent(
    name="ImageGenerator",
    model="gemini-2.0-flash-exp",
    instruction="""당신은 ImageGenerator입니다.
제공된 프롬프트를 바탕으로 이미지를 만들기 위해 generate_image 도구를 사용하세요.
""",
    description="도구를 사용해 이미지를 생성합니다.",
    tools=[generate_image]
)

In [12]:
image_tool = agent_tool.AgentTool(
    agent=image_generator_agent,
    # description="이미지를 생성할 때 사용하세요. 입력은 원하는 이미지를 설명하는 상세한 프롬프트여야 합니다."
)

artist_agent = LlmAgent(
    name="Artist",
    model="gemini-2.0-flash-exp",
    instruction="""당신은 창의적인 아티스트입니다.
먼저, 창의적인 이미지 프롬프트를 고안하세요.
그 다음, 도구처럼 ImageGenerator 에이전트를 사용해 이미지를 생성하세요.
""",
    description="프롬프트를 고안하고 이미지를 생성하여 작품을 만듭니다.",
    tools=[image_tool]
)